# Симплекс метод

In [1]:
data class Constraint(
    val coefficients: List<Double>,
    val sign: Sign,
    val b: Double
)

enum class Sign {
    EQ, LE, GE
}

In [2]:
data class ObjectiveFunction(
    val coefficients: List<Double>,
    val goal: Goal
)

enum class Goal {
    MAX, MIN
}


In [3]:
enum class TableauResult(val text: String) {
    OPTIMAL("Решение оптимальное"),
    NOT_OPTIMAL("Решение не оптимальное"),
    UNBOUNDED("Функция неограничена"),
    NO_SOLUTION("Система не имеет решения"),
    NEGATIVE_B("В таблице есть отрицательные свободные коэффициенты"),
}

In [4]:
data class PivotElement(
    val row: Int,
    val col: Int
)

In [5]:
data class Answer(
    val vars: List<Double>,
    val f: Double
)

In [6]:
data class SimplexTableau(
    val tableau: List<List<Double>>,
    val basis: List<Int>,
    val pivotElement: PivotElement? = null,
    val tableauState: TableauResult? = null,
    val answer: Answer? = null
) {
    override fun toString() = buildString {
        append("Базис\t")
        for (i in 0..<tableau.first().size - 1) append("x${i + 1}\t\t")
        append("B\n")
        for (i in tableau.indices) {
            if (i < basis.size) append("x${basis[i]}\t\t") else append("F\t\t")
            tableau[i].forEach { append("%.2f\t".format(it)) }
            append("\n")
        }
        tableauState?.let {
            append(tableauState.text)
        }
        append("\n")
        pivotElement?.let {
            append("Разрещающий элемент: [${pivotElement.col}][${pivotElement.row}]")
        }
        answer?.let {
            it.vars.forEachIndexed { i, value -> append("x${i + 1} = %.2f\t".format(value))  }
            append("\n")
            append("%.2f".format(it.f))
        }
    }
}


In [8]:
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow

In [9]:
class Simplex(
    val objectiveFunction: ObjectiveFunction,
    val constraints: List<Constraint>,
    val numVars: Int
) {
    private val tableau: MutableList<MutableList<Double>> = mutableListOf()
    private val basis: MutableList<Int> = MutableList(constraints.size) { 0 }
    private val c = objectiveFunction.coefficients
    private val goal = objectiveFunction.goal
    private val addVars = constraints.count { it.sign != Sign.EQ }
    private val countCol = numVars + addVars + 1
    private val countRow = basis.size + 1

    init {
        if (!constraints.all { it.coefficients.size == numVars }) {
            throw Exception()
        }
        formTableau()
    }

    private fun formTableau() {
        convertToCanonForm()
        formInitialBasis()
        addRow(objectiveFunction.coefficients.map { -it }, 0.0)
    }

    /*
    Формирование начального базиса
     */
    private fun formInitialBasis() {
        var countBasisVars = 0
        val maxCountBasisVars = basis.size
        for (i in 0..<countCol - 1) {
            if (countBasisVars == maxCountBasisVars) break
            val currentCol = tableau.map { it[i] }
            val countNonZero = currentCol.count { it != 0.0 }
            if (countNonZero == 1) {
                val indexRow = currentCol.indexOfFirst { it != 0.0 }
                if (currentCol[indexRow] == 1.0) {
                    basis[indexRow] = i + 1
                } else {
                    basis[indexRow] = i + 1
                    tableau[indexRow] = tableau[indexRow].map { it / currentCol.first() }.toMutableList()
                    println(tableau[indexRow])
                }
                countBasisVars++
            }
        }
        for (i in 0..<countCol - 1) {
            if (countBasisVars == maxCountBasisVars) break
            val currentCol = tableau.map { it[i] }
            val countNonZero = currentCol.count { it == 0.0 }
            if (countNonZero == 0) {
                val indexBasis = basis.indexOfFirst { it == 0 }
                basis[indexBasis] = i + 1
                tableau[indexBasis] = tableau[indexBasis].map { it / currentCol[indexBasis] }.toMutableList()
                tableau.forEachIndexed { row, _ ->
                    if (row != indexBasis) {
                        tableau[row] = tableau[row].mapIndexed{ col, it ->
                            it - tableau[indexBasis][col] * tableau[row][i] }.toMutableList()
                    }
                }
                countBasisVars++
            }
        }
    }

    /*
    Преобразование в каноническую форму
     */
    private fun convertToCanonForm() {
        var indexAddVar = numVars - 1
        var coefficients: List<Double>
        var b: Double
        for (constraint in constraints) {
            when(constraint.sign) {
                Sign.GE -> {
                    coefficients = constraint.coefficients.map { -it }
                    b = -constraint.b
                    indexAddVar++
                    addRow(coefficients, b, indexAddVar)
                }
                Sign.LE -> {
                    coefficients = constraint.coefficients
                    b = constraint.b
                    indexAddVar++
                    addRow(coefficients, b, indexAddVar)
                }
                Sign.EQ -> {
                    coefficients = constraint.coefficients
                    b = constraint.b
                    addRow(coefficients, b)
                }
            }
        }
    }

    /*
    Добавление строки в таблицу
     */
    private fun addRow(coefficients: List<Double>, b: Double, indexAddVar: Int? = null) {
        val row: MutableList<Double> = MutableList(countCol) { 0.0 }
        for (i in 0..<countCol) {
            if (i < coefficients.size) row[i] = coefficients[i]
            else if (indexAddVar != null && i == indexAddVar) row[i] = 1.0
            else if (i == countCol - 1) row[i] = b
            else continue
        }
        tableau.add(row)
    }

    /*
    Проверка на наличие отрицательных свободных коэффициентов (b)
     */
    private fun checkNegativeB(): Boolean = tableau.dropLast(1).any {it.last() < 0 }

    /*
    Перерасчёт таблицы
     */
    private fun recalculate(pivotElement: PivotElement) {
        val pivotRow = pivotElement.row
        val pivotCol = pivotElement.col
        basis[pivotElement.row] = pivotElement.col + 1
        for (i in 0..<countRow) {
            for (j in 0..<countCol) {
                if (i != pivotRow && j != pivotCol) {
                    tableau[i][j] = tableau[i][j] - tableau[i][pivotCol] * tableau[pivotRow][j] / tableau[pivotRow][pivotCol]
                }
            }
        }
        for (j in 0..<countCol) {
            if (j != pivotCol) {
                tableau[pivotRow][j] = tableau[pivotRow][j] / tableau[pivotRow][pivotCol]
            }
        }
        for (i in 0..<countRow) {
            if (i != pivotRow) {
                tableau[i][pivotCol] = 0.0
            } else {
                tableau[i][pivotCol] = 1.0
            }
        }
    }

    /*
    Поиск разрещающего элемента
     */
    private fun findPivotElement(containsNegativeB: Boolean = false): PivotElement? {
        return if (containsNegativeB) {
            val pivotRow: Int = findPivotRow()
            val pivotCol: Int? = findPivotCol(pivotRow)
            pivotCol?.let { PivotElement(row = pivotRow, col = pivotCol) }
        } else {
            val pivotCol: Int = findPivotCol()
            val pivotRow: Int? = findPivotRow(pivotCol)
            pivotRow?.let { PivotElement(row = pivotRow, col = pivotCol) }
        }
    }

    /*
    Поиск разрещающей строки
     */
    private fun findPivotRow(): Int {
        val b = tableau.dropLast(1).map { it.last() }
        val min = b.filter { it < 0 }.min()
        return b.indexOf(min)
    }

    private fun findPivotRow(pivotCol: Int): Int? {
        val b = tableau.dropLast(1).map { it.last() / it[pivotCol] }
        val min: Double? = b.filter { it > 0 }.minOrNull()
        return min?.let { b.indexOf(min) }
    }

    /*
    Поиск разрещающего столбца
     */
    private fun findPivotCol(): Int {
        return when (goal) {
            Goal.MAX -> {
                val min = tableau.last().dropLast(1).filter { it < 0 }.min()
                tableau.last().dropLast(1).indexOf(min)
            }
            Goal.MIN -> {
                val max = tableau.last().dropLast(1).filter { it > 0 }.max()
                tableau.last().dropLast(1).indexOf(max)
            }
        }
    }

    private fun findPivotCol(minRow: Int): Int? {
        val b = tableau[minRow].dropLast(1)
        val min = b.filter { it < 0 }.minOrNull()
        return min?.let { b.indexOf(min) }
    }

    /*
    Проверка на оптимальность
     */
    private fun checkOptimality(): Boolean {
        return when (goal) {
            Goal.MAX -> tableau.last().dropLast(1).all { it >= 0 }
            Goal.MIN -> tableau.last().dropLast(1).all { it <= 0 }
        }
    }

    /*
    Получение ответа
     */
    private fun getAnswer(): Answer {
        val vars: MutableList<Double> = mutableListOf()
        val f = tableau.last().last()
        for (i in c.indices) {
            val x = i + 1
            val value = if (x in basis) tableau[basis.indexOf(x)].last() else 0.0
            vars.add(value)
        }
        return Answer(vars, f)
    }

    /*
    Проверка таблицы
     */
    private fun checkTableau(): SimplexTableau {
        val simplexTableau = SimplexTableau(tableau, basis)
        if (checkNegativeB()) {
            val pivotElement = findPivotElement(checkNegativeB())
                ?: return simplexTableau.copy(tableauState = TableauResult.NO_SOLUTION)
            return simplexTableau.copy(pivotElement = pivotElement, tableauState = TableauResult.NEGATIVE_B)
        } else {
            if (checkOptimality()) {
                val answer = getAnswer()
                return simplexTableau.copy(tableauState = TableauResult.OPTIMAL, answer = answer)
            } else {
                val pivotElement = findPivotElement()
                    ?: return simplexTableau.copy(tableauState = TableauResult.UNBOUNDED)
                return simplexTableau.copy(pivotElement = pivotElement, tableauState = TableauResult.NOT_OPTIMAL)
            }
        }
    }

    fun solve(): Flow<SimplexTableau> = flow {
        var simplexTableau = checkTableau()
        var simplexResult = simplexTableau.tableauState
        emit(simplexTableau)
        while (simplexResult == TableauResult.NOT_OPTIMAL || simplexResult == TableauResult.NEGATIVE_B) {
            simplexTableau.pivotElement?.let { recalculate(it) }
            simplexTableau = checkTableau()
            simplexResult = simplexTableau.tableauState
            emit(simplexTableau)
        }
    }
}

In [13]:
suspend fun simplexSolve(simplex: Simplex) {
    simplex.solve().collect{ println(it) }
}

In [11]:
import kotlinx.coroutines.runBlocking

### Симплекс-метод: система не имеет ни одного решения

![1 пример](screenshots/1.png "1 пример")

In [15]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(1.0, -1.0), Goal.MAX)
    val constraints = listOf(
        Constraint(listOf(-2.0, 3.0), Sign.GE, 9.0),
        Constraint(listOf(1.0, -2.0), Sign.GE, 2.0),
        Constraint(listOf(1.0, 1.0), Sign.LE, 8.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		B
x3		2,00	-3,00	1,00	0,00	0,00	-9,00	
x4		-1,00	2,00	0,00	1,00	0,00	-2,00	
x5		1,00	1,00	0,00	0,00	1,00	8,00	
F		-1,00	1,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [1][0]
Базис	x1		x2		x3		x4		x5		B
x2		-0,67	1,00	-0,33	-0,00	-0,00	3,00	
x4		0,33	0,00	0,67	1,00	0,00	-8,00	
x5		1,67	0,00	0,33	0,00	1,00	5,00	
F		-0,33	0,00	0,33	0,00	0,00	-3,00	
Система не имеет решения



### Симплекс-метод: функция неограничена
![2 пример](screenshots/2.png "2 пример")

In [16]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(1.0, 2.0), Goal.MAX)
    val constraints = listOf(
        Constraint(listOf(-2.0, 3.0), Sign.LE, 9.0),
        Constraint(listOf(1.0, -2.0), Sign.LE, 2.0),
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		B
x3		-2,00	3,00	1,00	0,00	9,00	
x4		1,00	-2,00	0,00	1,00	2,00	
F		-1,00	-2,00	0,00	0,00	0,00	
Решение не оптимальное
Разрещающий элемент: [1][0]
Базис	x1		x2		x3		x4		B
x2		-0,67	1,00	0,33	0,00	3,00	
x4		-0,33	0,00	0,67	1,00	8,00	
F		-2,33	0,00	0,67	0,00	6,00	
Функция неограничена



### Симплекс-метод: вариант 11а
![3 пример](screenshots/3.png "3 пример") ![3 пример сравнение](screenshots/3a.png "3 пример сравнение")

In [18]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(7.0, 3.0), Goal.MAX)
    val constraints = listOf(
        Constraint(listOf(3.0, 2.0), Sign.LE, 8.0),
        Constraint(listOf(1.0, 2.0), Sign.LE, 6.0),
        Constraint(listOf(2.0, 3.0), Sign.GE, 3.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		B
x3		3,00	2,00	1,00	0,00	0,00	8,00	
x4		1,00	2,00	0,00	1,00	0,00	6,00	
x5		-2,00	-3,00	0,00	0,00	1,00	-3,00	
F		-7,00	-3,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [1][2]
Базис	x1		x2		x3		x4		x5		B
x3		1,67	0,00	1,00	0,00	0,67	6,00	
x4		-0,33	0,00	0,00	1,00	0,67	4,00	
x2		0,67	1,00	-0,00	-0,00	-0,33	1,00	
F		-5,00	0,00	0,00	0,00	-1,00	3,00	
Решение не оптимальное
Разрещающий элемент: [0][2]
Базис	x1		x2		x3		x4		x5		B
x3		0,00	-2,50	1,00	0,00	1,50	3,50	
x4		0,00	0,50	0,00	1,00	0,50	4,50	
x1		1,00	1,50	-0,00	-0,00	-0,50	1,50	
F		0,00	7,50	0,00	0,00	-3,50	10,50	
Решение не оптимальное
Разрещающий элемент: [4][0]
Базис	x1		x2		x3		x4		x5		B
x5		0,00	-1,67	0,67	0,00	1,00	2,33	
x4		0,00	1,33	-0,33	1,00	0,00	3,33	
x1		1,00	0,67	0,33	0,00	0,00	2,67	
F		0,00	1,67	2,33	0,00	0,00	18,67	
Решение оптимальное
x1 = 2,67	x2 = 0,00	
18,67


### Симплекс-метод: вариант 11б
![4 пример](screenshots/4.png "4 пример") ![4 пример сравнение](screenshots/4a.png "4 пример сравнение")

In [19]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(2.0, 1.0), Goal.MIN)
    val constraints = listOf(
        Constraint(listOf(1.0, 2.0), Sign.LE, 4.0),
        Constraint(listOf(2.0, 1.0), Sign.LE, 6.0),
        Constraint(listOf(2.0, 1.0), Sign.GE, 2.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		B
x3		1,00	2,00	1,00	0,00	0,00	4,00	
x4		2,00	1,00	0,00	1,00	0,00	6,00	
x5		-2,00	-1,00	0,00	0,00	1,00	-2,00	
F		-2,00	-1,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [0][2]
Базис	x1		x2		x3		x4		x5		B
x3		0,00	1,50	1,00	0,00	0,50	3,00	
x4		0,00	0,00	0,00	1,00	1,00	4,00	
x1		1,00	0,50	-0,00	-0,00	-0,50	1,00	
F		0,00	0,00	0,00	0,00	-1,00	2,00	
Решение оптимальное
x1 = 1,00	x2 = 0,00	
2,00


### Симплекс-метод: вариант 11в
![5 пример](screenshots/5.png "5 пример") ![5 пример сравнение](screenshots/5a.png "5 пример сравнение")

In [21]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(2.0, 8.0), Goal.MAX)
    val constraints = listOf(
        Constraint(listOf(1.0, 2.0), Sign.GE, 8.0),
        Constraint(listOf(1.0, 1.0), Sign.GE, 16.0),
        Constraint(listOf(3.0, 1.0), Sign.GE, 3.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		B
x3		-1,00	-2,00	1,00	0,00	0,00	-8,00	
x4		-1,00	-1,00	0,00	1,00	0,00	-16,00	
x5		-3,00	-1,00	0,00	0,00	1,00	-3,00	
F		-2,00	-8,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [0][1]
Базис	x1		x2		x3		x4		x5		B
x3		0,00	-1,00	1,00	-1,00	0,00	8,00	
x1		1,00	1,00	-0,00	-1,00	-0,00	16,00	
x5		0,00	2,00	0,00	-3,00	1,00	45,00	
F		0,00	-6,00	0,00	-2,00	0,00	32,00	
Решение не оптимальное
Разрещающий элемент: [1][1]
Базис	x1		x2		x3		x4		x5		B
x3		1,00	0,00	1,00	-2,00	0,00	24,00	
x2		1,00	1,00	-0,00	-1,00	-0,00	16,00	
x5		-2,00	0,00	0,00	-1,00	1,00	13,00	
F		6,00	0,00	0,00	-8,00	0,00	128,00	
Функция неограничена



### Симплекс-метод: вариант 11г
![6 пример](screenshots/6.png "6 пример") ![6 пример сравнение](screenshots/6a.png "6 пример сравнение")

In [22]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(6.0, 2.0), Goal.MIN)
    val constraints = listOf(
        Constraint(listOf(6.0, 2.0), Sign.LE, 8.0),
        Constraint(listOf(1.0, 3.0), Sign.LE, 6.0),
        Constraint(listOf(1.0, 1.0), Sign.GE, 16.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		B
x3		6,00	2,00	1,00	0,00	0,00	8,00	
x4		1,00	3,00	0,00	1,00	0,00	6,00	
x5		-1,00	-1,00	0,00	0,00	1,00	-16,00	
F		-6,00	-2,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [0][2]
Базис	x1		x2		x3		x4		x5		B
x3		0,00	-4,00	1,00	0,00	6,00	-88,00	
x4		0,00	2,00	0,00	1,00	1,00	-10,00	
x1		1,00	1,00	-0,00	-0,00	-1,00	16,00	
F		0,00	4,00	0,00	0,00	-6,00	96,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [1][0]
Базис	x1		x2		x3		x4		x5		B
x2		-0,00	1,00	-0,25	-0,00	-1,50	22,00	
x4		0,00	0,00	0,50	1,00	4,00	-54,00	
x1		1,00	0,00	0,25	0,00	0,50	-6,00	
F		0,00	0,00	1,00	0,00	0,00	8,00	
Система не имеет решения



### Симплекс-метод: пример 1
![7 пример](screenshots/7.png "7 пример") ![7 пример сравнение](screenshots/7a.png "7 пример сравнение")

In [25]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(2.0, 1.0), Goal.MAX)
    val constraints = listOf(
        Constraint(listOf(4.0, 6.0), Sign.GE, 20.0),
        Constraint(listOf(2.0, -5.0), Sign.GE, -27.0),
        Constraint(listOf(7.0, 5.0), Sign.LE, 63.0),
        Constraint(listOf(3.0, -2.0), Sign.LE, 23.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		x6		B
x3		-4,00	-6,00	1,00	0,00	0,00	0,00	-20,00	
x4		-2,00	5,00	0,00	1,00	0,00	0,00	27,00	
x5		7,00	5,00	0,00	0,00	1,00	0,00	63,00	
x6		3,00	-2,00	0,00	0,00	0,00	1,00	23,00	
F		-2,00	-1,00	0,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [1][0]
Базис	x1		x2		x3		x4		x5		x6		B
x2		0,67	1,00	-0,17	-0,00	-0,00	-0,00	3,33	
x4		-5,33	0,00	0,83	1,00	0,00	0,00	10,33	
x5		3,67	0,00	0,83	0,00	1,00	0,00	46,33	
x6		4,33	0,00	-0,33	0,00	0,00	1,00	29,67	
F		-1,33	0,00	-0,17	0,00	0,00	0,00	3,33	
Решение не оптимальное
Разрещающий элемент: [0][0]
Базис	x1		x2		x3		x4		x5		x6		B
x1		1,00	1,50	-0,25	-0,00	-0,00	-0,00	5,00	
x4		0,00	8,00	-0,50	1,00	0,00	0,00	37,00	
x5		0,00	-5,50	1,75	0,00	1,00	0,00	28,00	
x6		0,00	-6,50	0,75	0,00	0,00	1,00	8,00	
F		0,00	2,00	-0,50	0,00	0,00	0,00	10,00	
Решение не оптимальное
Разрещающий элемент: [2][3]
Базис	x1		x2		x3		x4		x5		x6		B
x1		1,00	-0,67	0,00	0,00	0,00	0,33	7,67	
x4		0,00	3,67	0,00	

### Симплекс-метод: пример 2
![8 пример](screenshots/8.png "7 пример") ![8 пример сравнение](screenshots/8a.png "8 пример сравнение")

In [26]:
runBlocking {
    val objectiveFunction = ObjectiveFunction(listOf(2.0, 1.0), Goal.MIN)
    val constraints = listOf(
        Constraint(listOf(4.0, 6.0), Sign.GE, 20.0),
        Constraint(listOf(2.0, -5.0), Sign.GE, -27.0),
        Constraint(listOf(7.0, 5.0), Sign.LE, 63.0),
        Constraint(listOf(3.0, -2.0), Sign.LE, 23.0)
    )
    val simplex = Simplex(
        objectiveFunction = objectiveFunction,
        constraints = constraints,
        numVars = 2
    )
    simplexSolve(simplex)
}

Базис	x1		x2		x3		x4		x5		x6		B
x3		-4,00	-6,00	1,00	0,00	0,00	0,00	-20,00	
x4		-2,00	5,00	0,00	1,00	0,00	0,00	27,00	
x5		7,00	5,00	0,00	0,00	1,00	0,00	63,00	
x6		3,00	-2,00	0,00	0,00	0,00	1,00	23,00	
F		-2,00	-1,00	0,00	0,00	0,00	0,00	0,00	
В таблице есть отрицательные свободные коэффициенты
Разрещающий элемент: [1][0]
Базис	x1		x2		x3		x4		x5		x6		B
x2		0,67	1,00	-0,17	-0,00	-0,00	-0,00	3,33	
x4		-5,33	0,00	0,83	1,00	0,00	0,00	10,33	
x5		3,67	0,00	0,83	0,00	1,00	0,00	46,33	
x6		4,33	0,00	-0,33	0,00	0,00	1,00	29,67	
F		-1,33	0,00	-0,17	0,00	0,00	0,00	3,33	
Решение оптимальное
x1 = 0,00	x2 = 3,33	
3,33
